In [36]:
# 计算特定副本中各个门派的DPS。
# 与之前的版本不同，每个不同的玩家只会计算一次。

import numpy as np
import pymysql
import configparser
import json
import time
import matplotlib.pyplot as plt

In [37]:
ip = "139.199.102.41"
config = configparser.RawConfigParser()
config.readfp(open('../settings.cfg'))

dbname = config.get('jx3bla', 'username')
dbpwd = config.get('jx3bla', 'password')
db = pymysql.connect(host=ip,user=dbname,password=dbpwd,database="jx3bla",port=3306,charset='utf8')
cursor = db.cursor()

C:\Users\jinhao\AppData\Local\Temp/ipykernel_251044/2612104811.py:3: DeprecationWarning: This method will be removed in future versions.  Use 'parser.read_file()' instead.
  config.readfp(open('../settings.cfg'))


In [38]:
sql = """SELECT * FROM ReplayProStat"""
cursor.execute(sql)
result = cursor.fetchall()

In [39]:
print(result[0])

('梦江南', '小宴离', 'xiangzhi', 0, '2021-11-10', '25人英雄雷域大泽', '尤珈罗摩', '0014decf2ce9b7f53f95abdca84e3a6e', 497, b"{'dps': {'table': [{'shieldRate': 0.0, 'name': '\xe6\x95\xa3\xe5\xa6\xb9', 'damage': 504121, 'occ': '211', 'shieldBreak': 0}, {'shieldRate': 0.0, 'name': '\xe5\xa7\x9c\xe7\xa7\x83', 'damage': 403075, 'occ': '1', 'shieldBreak': 0}, {'shieldRate': 0.0, 'name': '\xe6\x98\xa5\xe5\xa4\x8f', 'damage': 370625, 'occ': '23', 'shieldBreak': 0}, {'shieldRate': 0.0, 'name': '\xe6\x98\x9f\xe6\xb2\x89\xe5\xad\x90', 'damage': 320345, 'occ': '7', 'shieldBreak': 0}, {'shieldRate': 0.0, 'name': '\xe7\xa7\xa6\xe6\xb8\x85\xe7\xba\xb2', 'damage': 318820, 'occ': '212', 'shieldBreak': 0}, {'shieldRate': 0.0, 'name': '\xe4\xb9\xb0\xe4\xba\x86\xe4\xb8\xaa\xe4\xbd\x9b\xe5\x86\xb7', 'damage': 314775, 'occ': '24', 'shieldBreak': 0}, {'shieldRate': 0.0, 'name': '\xe5\xb0\x8f\xe5\xae\xb4\xe6\xa2\xa8', 'damage': 304563, 'occ': '2', 'shieldBreak': 0}, {'shieldRate': 0.0, 'name': '\xe6\x95\xa3\xe4\xbb\x99@\xe7\x

In [40]:
def getSingle(t):
    t = t.decode('utf-8')
    t = t.replace("'", '"').replace("\t", "\\t").replace("\n", "\\n")
    j = json.loads(t)
    result = {}
    # 治疗量A1
    result["heal"] = 0
    for record in j["healer"]["table"]:
        name = record["name"]
        if name == j["overall"]["playerID"]:
            result["heal"] = record["healEff"]
            break
    # 盾数量A2
    result["numShield"] = j["skill"]["meihua"]["num"]
    # 徵数量A3
    result["numZhi"] = j["skill"]["zhi"]["num"]
    # 宫数量A4
    result["numGong"] = j["skill"]["gong"]["num"]
    # A类总计

    # 统计分B
    # 盾覆盖率B1
    result["coverShield"] = j["skill"]["meihua"]["cover"]
    # HOT覆盖率B2
    result["coverHot"] = (j["skill"]["shang"]["cover"] + j["skill"]["jue"]["cover"]) / 2
    # B类总计

    # 操作分C
    # 战斗效率C1
    result["efficiency"] = j["skill"]["general"]["efficiency"]
    # 盾延迟C2
    result["delayShield"] = j["skill"]["meihua"]["delay"]
    # 徵延迟C3
    result["delayZhi"] = j["skill"]["zhi"]["delay"]
    # 宫延迟C4
    result["delayGong"] = j["skill"]["gong"]["delay"]
    # C类总计

    # 总分
    return result


    

In [41]:
text = result[0][9]
r = getSingle(text)
print(r)

{'heal': 76461, 'numShield': 1, 'numZhi': 370, 'numGong': 1218, 'coverShield': 0.0, 'coverHot': 0.71925, 'efficiency': 0.8546920521997118, 'delayShield': 646, 'delayZhi': 11, 'delayGong': 184}


In [42]:
def getStatistics(map, boss, results):
    print("*****************")
    print("地图：", map)
    print("首领：", boss)
    std = results[0]
    for key in std:
        print("[%s]"%key)
        l = []
        for line in results:
            l.append(line[key])
        L = np.asarray(l)
        print("数量：", len(l))
        print("平均数：", L.mean())
        print("最小值：", L.min())
        print("25%分位数：", np.quantile(L,0.25,interpolation='lower'))
        print("中位数：", np.median(L))
        print("75%分位数：", np.quantile(L,0.75,interpolation='higher'))
        print("最大值：", L.max())
        
        

In [43]:
mapList = ["25人普通雷域大泽", "25人英雄雷域大泽"]
bossList = ["巨型尖吻凤", "桑乔", "悉达罗摩", "尤珈罗摩", "月泉淮", "乌蒙贵"]

for map in mapList:
    for boss in bossList:
        results = []
        for line in result:
            if line[5] == map and line[6] == boss:
                raw = line[9]
                res = getSingle(raw)
                results.append(res)
        getStatistics(map, boss, results)
            

*****************
地图： 25人普通雷域大泽
首领： 巨型尖吻凤
[heal]
数量： 65
平均数： 4129.569230769231
最小值： 0
25%分位数： 2072
中位数： 3142.0
75%分位数： 5294
最大值： 17381
[numShield]
数量： 65
平均数： 54.58461538461538
最小值： 0
25%分位数： 44
中位数： 61.0
75%分位数： 71
最大值： 114
[numZhi]
数量： 65
平均数： 152.6769230769231
最小值： 0
25%分位数： 10
中位数： 70.0
75%分位数： 212
最大值： 1055
[numGong]
数量： 65
平均数： 2.8923076923076922
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 104
[coverShield]
数量： 65
平均数： 0.4564969230769231
最小值： 0.0
25%分位数： 0.4241
中位数： 0.5524
75%分位数： 0.6069
最大值： 0.883
[coverHot]
数量： 65
平均数： 106323384615384.61
最小值： 0.0
25%分位数： 0.0
中位数： 0.0018
75%分位数： 0.00895
最大值： 6694885000000000.0
[efficiency]
数量： 65
平均数： 0.5804238238616023
最小值： 0.0
25%分位数： 0.5265636914343809
中位数： 0.6398253222823842
75%分位数： 0.736128724872655
最大值： 0.876547736377733
[delayShield]
数量： 65
平均数： 261.66153846153844
最小值： 0
25%分位数： 174
中位数： 273.0
75%分位数： 368
最大值： 549
[delayZhi]
数量： 65
平均数： 110.26153846153846
最小值： 0
25%分位数： 27
中位数： 75.0
75%分位数： 123
最大值： 728
[delayGong]
数量： 65
平均数： 32.96923076923

25%分位数： 67502
中位数： 76590.0
75%分位数： 87567
最大值： 132969
[numShield]
数量： 167
平均数： 4.880239520958084
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 297
[numZhi]
数量： 167
平均数： 63.550898203592816
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 4616
[numGong]
数量： 167
平均数： 658.5748502994012
最小值： 0
25%分位数： 221
中位数： 631.0
75%分位数： 1084
最大值： 1454
[coverShield]
数量： 167
平均数： 0.005226946107784431
最小值： 0.0
25%分位数： 0.0
中位数： 0.0
75%分位数： 0.0
最大值： 0.277
[coverHot]
数量： 167
平均数： 0.6328976047904191
最小值： 0.0
25%分位数： 0.58925
中位数： 0.6488
75%分位数： 0.70655
最大值： 1.3286
[efficiency]
数量： 167
平均数： 0.7200382482433043
最小值： 0.13334834793594988
25%分位数： 0.6812930239771159
中位数： 0.7530931826419316
75%分位数： 0.7950011511453894
最大值： 1.189261791456259
[delayShield]
数量： 167
平均数： 10.119760479041917
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 646
[delayZhi]
数量： 167
平均数： 6.62874251497006
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 283
[delayGong]
数量： 167
平均数： 247.562874251497
最小值： 0
25%分位数： 202
中位数： 235.0
75%分位数： 284
最大值： 735
*****************
地图： 25人英雄雷

In [17]:
str = """*****************
地图： 25人普通雷域大泽
首领： 巨型尖吻凤
[heal]
数量： 65
平均数： 4129.569230769231
最小值： 0
25%分位数： 2072
中位数： 3142.0
75%分位数： 5294
最大值： 17381
[numShield]
数量： 65
平均数： 54.58461538461538
最小值： 0
25%分位数： 44
中位数： 61.0
75%分位数： 71
最大值： 114
[numZhi]
数量： 65
平均数： 152.6769230769231
最小值： 0
25%分位数： 10
中位数： 70.0
75%分位数： 212
最大值： 1055
[numGong]
数量： 65
平均数： 2.8923076923076922
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 104
[coverShield]
数量： 65
平均数： 0.4564969230769231
最小值： 0.0
25%分位数： 0.4241
中位数： 0.5524
75%分位数： 0.6069
最大值： 0.883
[coverHot]
数量： 65
平均数： 106323384615384.61
最小值： 0.0
25%分位数： 0.0
中位数： 0.0018
75%分位数： 0.00895
最大值： 6694885000000000.0
[efficiency]
数量： 65
平均数： 0.5804238238616023
最小值： 0.0
25%分位数： 0.5265636914343809
中位数： 0.6398253222823842
75%分位数： 0.736128724872655
最大值： 0.876547736377733
[delayShield]
数量： 65
平均数： 261.66153846153844
最小值： 0
25%分位数： 174
中位数： 273.0
75%分位数： 368
最大值： 549
[delayZhi]
数量： 65
平均数： 110.26153846153846
最小值： 0
25%分位数： 27
中位数： 75.0
75%分位数： 123
最大值： 728
[delayGong]
数量： 65
平均数： 32.96923076923077
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 970
*****************
地图： 25人普通雷域大泽
首领： 桑乔
[heal]
数量： 45
平均数： 3150.511111111111
最小值： 0
25%分位数： 1520
中位数： 2581.0
75%分位数： 4302
最大值： 8804
[numShield]
数量： 45
平均数： 64.2
最小值： 0
25%分位数： 56
中位数： 65.0
75%分位数： 74
最大值： 106
[numZhi]
数量： 45
平均数： 307.15555555555557
最小值： 0
25%分位数： 80
中位数： 325.0
75%分位数： 464
最大值： 1366
[numGong]
数量： 45
平均数： 0.8222222222222222
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 30
[coverShield]
数量： 45
平均数： 0.7125133333333333
最小值： 0.0
25%分位数： 0.7044
中位数： 0.7811
75%分位数： 0.8351
最大值： 0.9304
[coverHot]
数量： 45
平均数： 0.007963333333333333
最小值： 0.0
25%分位数： 0.0
中位数： 0.00545
75%分位数： 0.01035
最大值： 0.0401
[efficiency]
数量： 45
平均数： 0.5798280845026509
最小值： 0.05998224525540426
25%分位数： 0.4602246195476477
中位数： 0.5959565130937019
75%分位数： 0.7193695956983926
最大值： 0.8643346243222304
[delayShield]
数量： 45
平均数： 346.77777777777777
最小值： 0
25%分位数： 236
中位数： 370.0
75%分位数： 462
最大值： 598
[delayZhi]
数量： 45
平均数： 132.06666666666666
最小值： 0
25%分位数： 11
中位数： 90.0
75%分位数： 177
最大值： 698
[delayGong]
数量： 45
平均数： 60.888888888888886
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 959
*****************
地图： 25人普通雷域大泽
首领： 悉达罗摩
[heal]
数量： 65
平均数： 8983.307692307691
最小值： 2591
25%分位数： 4712
中位数： 7932.0
75%分位数： 11416
最大值： 21939
[numShield]
数量： 65
平均数： 109.81538461538462
最小值： 23
25%分位数： 84
中位数： 112.0
75%分位数： 138
最大值： 211
[numZhi]
数量： 65
平均数： 349.2153846153846
最小值： 0
25%分位数： 173
中位数： 301.0
75%分位数： 472
最大值： 1014
[numGong]
数量： 65
平均数： 0.18461538461538463
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 9
[coverShield]
数量： 65
平均数： 0.4506876923076923
最小值： 0.0
25%分位数： 0.3535
中位数： 0.4655
75%分位数： 0.5353
最大值： 0.7235
[coverHot]
数量： 65
平均数： 0.004033076923076923
最小值： 0.0
25%分位数： 0.0
中位数： 0.00265
75%分位数： 0.0071
最大值： 0.01985
[efficiency]
数量： 65
平均数： 0.6499619651836134
最小值： 0.44216932577879603
25%分位数： 0.5948625206618913
中位数： 0.6410648299097921
75%分位数： 0.6865450072293653
最大值： 0.8534145924702395
[delayShield]
数量： 65
平均数： 329.9384615384615
最小值： 122
25%分位数： 235
中位数： 317.0
75%分位数： 424
最大值： 649
[delayZhi]
数量： 65
平均数： 109.18461538461538
最小值： 0
25%分位数： 55
中位数： 84.0
75%分位数： 118
最大值： 743
[delayGong]
数量： 65
平均数： 52.13846153846154
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 2421
*****************
地图： 25人普通雷域大泽
首领： 尤珈罗摩
[heal]
数量： 103
平均数： 57848.28155339806
最小值： 7933
25%分位数： 43743
中位数： 56836.0
75%分位数： 72406
最大值： 129338
[numShield]
数量： 103
平均数： 4.796116504854369
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 176
[numZhi]
数量： 103
平均数： 101.32038834951456
最小值： 0
25%分位数： 0
中位数： 27.0
75%分位数： 90
最大值： 1819
[numGong]
数量： 103
平均数： 302.20388349514565
最小值： 0
25%分位数： 113
中位数： 231.0
75%分位数： 471
最大值： 868
[coverShield]
数量： 103
平均数： 0.010882524271844659
最小值： 0.0
25%分位数： 0.0
中位数： 0.0
75%分位数： 0.0
最大值： 0.3161
[coverHot]
数量： 103
平均数： 0.42334902912621364
最小值： 0.0
25%分位数： 0.3179
中位数： 0.40135
75%分位数： 0.5339499999999999
最大值： 0.8834
[efficiency]
数量： 103
平均数： 0.6222472046486751
最小值： 0.15064339307234714
25%分位数： 0.5323323542188707
中位数： 0.6246135295572585
75%分位数： 0.7293986470306508
最大值： 0.9312707710064938
[delayShield]
数量： 103
平均数： 28.611650485436893
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 1463
[delayZhi]
数量： 103
平均数： 73.06796116504854
最小值： 0
25%分位数： 0
中位数： 7.0
75%分位数： 95
最大值： 855
[delayGong]
数量： 103
平均数： 279.6504854368932
最小值： 0
25%分位数： 217
中位数： 274.0
75%分位数： 344
最大值： 661
*****************
地图： 25人普通雷域大泽
首领： 月泉淮
[heal]
数量： 61
平均数： 14029.327868852459
最小值： 648
25%分位数： 5180
中位数： 10281.0
75%分位数： 21187
最大值： 56785
[numShield]
数量： 61
平均数： 92.39344262295081
最小值： 0
25%分位数： 0
中位数： 115.0
75%分位数： 141
最大值： 199
[numZhi]
数量： 61
平均数： 560.5573770491803
最小值： 0
25%分位数： 65
中位数： 481.0
75%分位数： 889
最大值： 2073
[numGong]
数量： 61
平均数： 173.72131147540983
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 207
最大值： 1524
[coverShield]
数量： 61
平均数： 0.3619065573770492
最小值： 0.0
25%分位数： 0.0
中位数： 0.455
75%分位数： 0.5934
最大值： 0.7291
[coverHot]
数量： 61
平均数： 0.16988360655737705
最小值： 0.0
25%分位数： 0.00375
中位数： 0.00725
75%分位数： 0.34030000000000005
最大值： 1.0800999999999998
[efficiency]
数量： 61
平均数： 0.6396714746167287
最小值： 0.13303565355515273
25%分位数： 0.5639987615034909
中位数： 0.6394484176453862
75%分位数： 0.7526078538232545
最大值： 0.8433911164375151
[delayShield]
数量： 61
平均数： 226.49180327868854
最小值： 0
25%分位数： 0
中位数： 228.0
75%分位数： 352
最大值： 627
[delayZhi]
数量： 61
平均数： 96.95081967213115
最小值： 0
25%分位数： 24
中位数： 73.0
75%分位数： 108
最大值： 638
[delayGong]
数量： 61
平均数： 137.34426229508196
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 245
最大值： 792
*****************
地图： 25人普通雷域大泽
首领： 乌蒙贵
[heal]
数量： 57
平均数： 14115.701754385966
最小值： 980
25%分位数： 5490
中位数： 9922.0
75%分位数： 17161
最大值： 83271
[numShield]
数量： 57
平均数： 81.0
最小值： 0
25%分位数： 47
中位数： 77.0
75%分位数： 113
最大值： 231
[numZhi]
数量： 57
平均数： 490.7368421052632
最小值： 0
25%分位数： 93
中位数： 432.0
75%分位数： 771
最大值： 1833
[numGong]
数量： 57
平均数： 65.7719298245614
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 838
[coverShield]
数量： 57
平均数： 0.3645052631578948
最小值： 0.0
25%分位数： 0.2731
中位数： 0.407
75%分位数： 0.4991
最大值： 0.7119
[coverHot]
数量： 57
平均数： 0.07399649122807019
最小值： 0.0
25%分位数： 0.0019
中位数： 0.0053
75%分位数： 0.013749999999999998
最大值： 0.7665
[efficiency]
数量： 57
平均数： 0.555872642815165
最小值： 0.06859214436285017
25%分位数： 0.47823599502097736
中位数： 0.5429267230900567
75%分位数： 0.6679941661848949
最大值： 0.8266368937124432
[delayShield]
数量： 57
平均数： 347.280701754386
最小值： 0
25%分位数： 227
中位数： 361.0
75%分位数： 519
最大值： 682
[delayZhi]
数量： 57
平均数： 152.42105263157896
最小值： 0
25%分位数： 76
中位数： 102.0
75%分位数： 132
最大值： 1187
[delayGong]
数量： 57
平均数： 49.50877192982456
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 391
*****************
地图： 25人英雄雷域大泽
首领： 巨型尖吻凤
[heal]
数量： 108
平均数： 4488.064814814815
最小值： 0
25%分位数： 2259
中位数： 3614.5
75%分位数： 5358
最大值： 32575
[numShield]
数量： 108
平均数： 80.87962962962963
最小值： 0
25%分位数： 72
中位数： 92.0
75%分位数： 104
最大值： 136
[numZhi]
数量： 108
平均数： 252.4351851851852
最小值： 0
25%分位数： 44
中位数： 182.5
75%分位数： 400
最大值： 1330
[numGong]
数量： 108
平均数： 10.731481481481481
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 544
[coverShield]
数量： 108
平均数： 3.60838425925926
最小值： 0.0
25%分位数： 0.4782
中位数： 0.6427499999999999
75%分位数： 0.7105
最大值： 331.6111
[coverHot]
数量： 108
平均数： 324228564814814.9
最小值： 0.0
25%分位数： 0.0
中位数： 0.00255
75%分位数： 0.009600000000000001
最大值： 3.5016685e+16
[efficiency]
数量： 108
平均数： 0.6318660274358949
最小值： 0.0
25%分位数： 0.579312937824039
中位数： 0.6941223120425247
75%分位数： 0.7691605077165561
最大值： 0.8928874882424314
[delayShield]
数量： 108
平均数： 302.6388888888889
最小值： 0
25%分位数： 201
中位数： 278.5
75%分位数： 343
最大值： 2167
[delayZhi]
数量： 108
平均数： 112.98148148148148
最小值： 0
25%分位数： 37
中位数： 90.5
75%分位数： 155
最大值： 732
[delayGong]
数量： 108
平均数： 59.888888888888886
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 2163
*****************
地图： 25人英雄雷域大泽
首领： 桑乔
[heal]
数量： 88
平均数： 3562.0
最小值： 0
25%分位数： 1984
中位数： 3409.0
75%分位数： 4563
最大值： 14532
[numShield]
数量： 88
平均数： 72.70454545454545
最小值： 0
25%分位数： 62
中位数： 75.0
75%分位数： 88
最大值： 134
[numZhi]
数量： 88
平均数： 667.3636363636364
最小值： 0
25%分位数： 258
中位数： 626.0
75%分位数： 987
最大值： 2674
[numGong]
数量： 88
平均数： 10.579545454545455
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 785
[coverShield]
数量： 88
平均数： 0.7730545454545453
最小值： 0.0
25%分位数： 0.7432
中位数： 0.84215
75%分位数： 0.8832
最大值： 0.9349
[coverHot]
数量： 88
平均数： 0.013950000000000002
最小值： 0.0
25%分位数： 0.001
中位数： 0.005549999999999999
75%分位数： 0.012199999999999999
最大值： 0.57315
[efficiency]
数量： 88
平均数： 0.5019121887198618
最小值： 0.0031929166991559936
25%分位数： 0.409507458945196
中位数： 0.5305947618377735
75%分位数： 0.6172344428741121
最大值： 0.8390539911470052
[delayShield]
数量： 88
平均数： 319.27272727272725
最小值： 0
25%分位数： 270
中位数： 316.5
75%分位数： 376
最大值： 580
[delayZhi]
数量： 88
平均数： 102.125
最小值： 0
25%分位数： 65
中位数： 97.5
75%分位数： 135
最大值： 275
[delayGong]
数量： 88
平均数： 52.375
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 616
*****************
地图： 25人英雄雷域大泽
首领： 悉达罗摩
[heal]
数量： 158
平均数： 7054.2594936708865
最小值： 0
25%分位数： 4433
中位数： 6413.0
75%分位数： 8911
最大值： 46326
[numShield]
数量： 158
平均数： 145.9746835443038
最小值： 0
25%分位数： 105
中位数： 154.5
75%分位数： 199
最大值： 289
[numZhi]
数量： 158
平均数： 498.54430379746833
最小值： 0
25%分位数： 104
中位数： 412.5
75%分位数： 708
最大值： 2558
[numGong]
数量： 158
平均数： 9.917721518987342
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 1476
[coverShield]
数量： 158
平均数： 0.5449284810126582
最小值： 0.0
25%分位数： 0.4926
中位数： 0.5756
75%分位数： 0.6544
最大值： 0.8053
[coverHot]
数量： 158
平均数： 0.010492721518987341
最小值： 0.0
25%分位数： 0.0007
中位数： 0.0049250000000000006
75%分位数： 0.0097
最大值： 0.71605
[efficiency]
数量： 158
平均数： 0.6319837463431783
最小值： 0.014540120438519735
25%分位数： 0.5579761523249075
中位数： 0.6664883279666458
75%分位数： 0.7242158121950298
最大值： 0.8933440809591064
[delayShield]
数量： 158
平均数： 307.7974683544304
最小值： 0
25%分位数： 246
中位数： 300.0
75%分位数： 353
最大值： 1200
[delayZhi]
数量： 158
平均数： 98.73417721518987
最小值： 0
25%分位数： 55
中位数： 89.5
75%分位数： 128
最大值： 795
[delayGong]
数量： 158
平均数： 44.449367088607595
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 2167
*****************
地图： 25人英雄雷域大泽
首领： 尤珈罗摩
[heal]
数量： 167
平均数： 75771.68862275449
最小值： 2599
25%分位数： 67502
中位数： 76590.0
75%分位数： 87567
最大值： 132969
[numShield]
数量： 167
平均数： 4.880239520958084
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 297
[numZhi]
数量： 167
平均数： 63.550898203592816
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 4616
[numGong]
数量： 167
平均数： 658.5748502994012
最小值： 0
25%分位数： 221
中位数： 631.0
75%分位数： 1084
最大值： 1454
[coverShield]
数量： 167
平均数： 0.005226946107784431
最小值： 0.0
25%分位数： 0.0
中位数： 0.0
75%分位数： 0.0
最大值： 0.277
[coverHot]
数量： 167
平均数： 0.6328976047904191
最小值： 0.0
25%分位数： 0.58925
中位数： 0.6488
75%分位数： 0.70655
最大值： 1.3286
[efficiency]
数量： 167
平均数： 0.7200382482433043
最小值： 0.13334834793594988
25%分位数： 0.6812930239771159
中位数： 0.7530931826419316
75%分位数： 0.7950011511453894
最大值： 1.189261791456259
[delayShield]
数量： 167
平均数： 10.119760479041917
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 646
[delayZhi]
数量： 167
平均数： 6.62874251497006
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 283
[delayGong]
数量： 167
平均数： 247.562874251497
最小值： 0
25%分位数： 202
中位数： 235.0
75%分位数： 284
最大值： 735
*****************
地图： 25人英雄雷域大泽
首领： 月泉淮
[heal]
数量： 210
平均数： 14506.385714285714
最小值： 0
25%分位数： 8170
中位数： 10958.5
75%分位数： 16076
最大值： 66321
[numShield]
数量： 210
平均数： 115.95714285714286
最小值： 0
25%分位数： 46
中位数： 94.0
75%分位数： 168
最大值： 384
[numZhi]
数量： 210
平均数： 863.8571428571429
最小值： 0
25%分位数： 234
中位数： 678.0
75%分位数： 1171
最大值： 5298
[numGong]
数量： 210
平均数： 81.89047619047619
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 2182
[coverShield]
数量： 210
平均数： 0.32123999999999997
最小值： 0.0
25%分位数： 0.1917
中位数： 0.38239999999999996
75%分位数： 0.4594
最大值： 0.6953
[coverHot]
数量： 210
平均数： 0.07443714285714287
最小值： 0.0
25%分位数： 0.00085
中位数： 0.004
75%分位数： 0.007200000000000001
最大值： 0.8521
[efficiency]
数量： 210
平均数： 0.5899113326346733
最小值： -0.0
25%分位数： 0.5483920500876015
中位数： 0.6413397365152389
75%分位数： 0.7135263009948744
最大值： 0.8374498647299092
[delayShield]
数量： 210
平均数： 285.8142857142857
最小值： 0
25%分位数： 168
中位数： 287.5
75%分位数： 356
最大值： 2756
[delayZhi]
数量： 210
平均数： 75.44761904761904
最小值： 0
25%分位数： 39
中位数： 66.0
75%分位数： 109
最大值： 380
[delayGong]
数量： 210
平均数： 69.45238095238095
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 1519
*****************
地图： 25人英雄雷域大泽
首领： 乌蒙贵
[heal]
数量： 83
平均数： 8271.421686746988
最小值： 372
25%分位数： 3476
中位数： 6520.0
75%分位数： 11394
最大值： 25045
[numShield]
数量： 83
平均数： 121.57831325301204
最小值： 5
25%分位数： 100
中位数： 121.0
75%分位数： 139
最大值： 271
[numZhi]
数量： 83
平均数： 1040.7590361445782
最小值： 0
25%分位数： 288
中位数： 1009.0
75%分位数： 1465
最大值： 3725
[numGong]
数量： 83
平均数： 8.216867469879517
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 131
[coverShield]
数量： 83
平均数： 0.5077445783132529
最小值： 0.0
25%分位数： 0.4461
中位数： 0.5388
75%分位数： 0.6054
最大值： 0.7989
[coverHot]
数量： 83
平均数： 0.006468072289156626
最小值： 0.0
25%分位数： 0.00055
中位数： 0.00435
75%分位数： 0.00915
最大值： 0.04785
[efficiency]
数量： 83
平均数： 0.5016879471214885
最小值： 0.07507757322663026
25%分位数： 0.390003027326253
中位数： 0.492930203827484
75%分位数： 0.6544701732604371
最大值： 0.870996129634908
[delayShield]
数量： 83
平均数： 374.0240963855422
最小值： 0
25%分位数： 301
中位数： 367.0
75%分位数： 445
最大值： 700
[delayZhi]
数量： 83
平均数： 127.56626506024097
最小值： 0
25%分位数： 61
中位数： 89.0
75%分位数： 135
最大值： 1742
[delayGong]
数量： 83
平均数： 93.8433734939759
最小值： 0
25%分位数： 0
中位数： 0.0
75%分位数： 0
最大值： 2437"""

t = str.split('\n')
mmap = ""
boss = ""
attrib = ""
res = {}
tmp = [0, 0]
for line in t:
    if line.split(' ')[0] == "地图：":
        mmap = line.split(' ')[1]
    elif line.split(' ')[0] == "首领：":
        boss = line.split(' ')[1]
    elif line[0] == '[':
        attrib = line[1:-1]
        if attrib not in res:
            res[attrib] = {}
        if mmap not in res[attrib]:
            res[attrib][mmap] = {}
        if boss not in res[attrib][mmap]:
            res[attrib][mmap][boss] = []
    elif line[0] != '*':
        k = line.split(' ')[0]
        v = float(line.split(' ')[1])
        v = int(v*100)/100
        if k == "平均数：":
            v1 = 0
            v2 = v
            v3 = 0
        elif k == "25%分位数：":
            v1 = v
        elif k == "75%分位数：":
            v3 = v
            if 'delay' not in attrib:
                res[attrib][mmap][boss] = [[0, 0], [v1, 3], [v2, 6], [v3, 10]]
            else:
                res[attrib][mmap][boss] = [[1, 0], [v3, 3], [v2, 6], [v1, 10]]
            
for line in res:
    print(line)
    print(res[line])
            
        

heal
{'25人普通雷域大泽': {'巨型尖吻凤': [[0, 0], [2072.0, 3], [4129.56, 6], [5294.0, 10]], '桑乔': [[0, 0], [1520.0, 3], [3150.51, 6], [4302.0, 10]], '悉达罗摩': [[0, 0], [4712.0, 3], [8983.3, 6], [11416.0, 10]], '尤珈罗摩': [[0, 0], [43743.0, 3], [57848.28, 6], [72406.0, 10]], '月泉淮': [[0, 0], [5180.0, 3], [14029.32, 6], [21187.0, 10]], '乌蒙贵': [[0, 0], [5490.0, 3], [14115.7, 6], [17161.0, 10]]}, '25人英雄雷域大泽': {'巨型尖吻凤': [[0, 0], [2259.0, 3], [4488.06, 6], [5358.0, 10]], '桑乔': [[0, 0], [1984.0, 3], [3562.0, 6], [4563.0, 10]], '悉达罗摩': [[0, 0], [4433.0, 3], [7054.25, 6], [8911.0, 10]], '尤珈罗摩': [[0, 0], [67502.0, 3], [75771.68, 6], [87567.0, 10]], '月泉淮': [[0, 0], [8170.0, 3], [14506.38, 6], [16076.0, 10]], '乌蒙贵': [[0, 0], [3476.0, 3], [8271.42, 6], [11394.0, 10]]}}
numShield
{'25人普通雷域大泽': {'巨型尖吻凤': [[0, 0], [44.0, 3], [54.58, 6], [71.0, 10]], '桑乔': [[0, 0], [56.0, 3], [64.2, 6], [74.0, 10]], '悉达罗摩': [[0, 0], [84.0, 3], [109.81, 6], [138.0, 10]], '尤珈罗摩': [[0, 0], [0.0, 3], [4.79, 6], [0.0, 10]], '月泉淮': [[0, 0], [